In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install apyori
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from apyori import apriori
from collections import Counter
import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('fivethirtyeight')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# loading the dataframe

In [ ]:
df = pd.read_csv('/kaggle/input/groceries-dataset/Groceries_dataset.csv')
df.head()
df3 = df.sort_values(by = 'Member_number')
df[df['Member_number'] == 3831]

# Basic initial analysis

basic information

In [ ]:
#['Member_number', 'Date', 'itemDescription']
print('basic info')
print(df.info())
print('no nulls in dataframe')
print(df.isnull().sum())
print('count of appearences of a member in data ')
print(df['Member_number'].value_counts())
print('count of appearences of an item in data ')
print(df['itemDescription'].value_counts())

# creating a dataframe summarising all the items a customer bought on a perticular day

In [ ]:
#['Member_number', 'Date', 'itemDescription']
date_group = df.groupby('Date')
df2 = pd.DataFrame(columns=('Date', 'cus_id', 'Items')) 
for date in date_group:
    cus = []
    cus = date_group['Member_number'].unique().loc[date[0]]
    for customer in cus:
        filt = ((df['Date'] == date[0]) & (df['Member_number'] == customer))
        item_str = ','.join(df.loc[filt,'itemDescription'])
        df2 = df2.append({'Date':date[0], 'cus_id':customer, 'Items':item_str},ignore_index = True)    

# Before market basket analysis, let's do some visual analysis

**Top-10 sold data in all transactions**

In [ ]:
all_items = (','.join(df2['Items'])).split(',')
item_count_dict = Counter(all_items)
top_items = item_count_dict.most_common(10)
top_items_lst = list(map(lambda x : x[0],top_items))
top_items_lst

**Plotting the distribution of top items over the time**

**Number of units sold mothly**

In [ ]:
df3 = df2.copy()
df3['TS'] = pd.to_datetime(df3['Date'], format='%d-%m-%Y')
#df3.set_index('TS',inplace=True)
# adding new columns for top item counts
for items in top_items_lst:
    df3[items] = 0
#df3.head()    

for x in range(len(df3)):
    for item in top_items_lst:
        df3.loc[x,item] = df3.loc[x,'Items'].count(item)
df3.set_index('TS',inplace = True)        
df3.drop(columns=['cus_id','Date','Items'],inplace = True)       
monthly_sell = df3.resample('M').sum()
monthly_sell.head()        

**Monthly variation of items**

In [ ]:
colour_list = ['blue','green','red','orange','purple','yellow','brown','cyan','olive','gray','magenta']
plt.figure(figsize=(15,10))
for i,items in enumerate(top_items_lst):
    plt.plot(monthly_sell[items],label = items,color = colour_list[i])
plt.xlabel('Time')
plt.ylabel('Units sold')
plt.title('Variation of units sold')
plt.legend()
plt.show()

# creating list of transactions, suitable for apriori

In [ ]:
transactions = []
for tran in df2['Items']:
    temp = tran.split(',')
    transactions.append(temp)  

# Data is ready for, creating the model and training it.

In [ ]:
association_rules = apriori(transactions,min_support=0.00030,min_confidence=0.01,min_lift=3,min_length=2)
association_result = list(association_rules)
association_result